# Introduction
In this guide, we will walk you through building a powerful semantic search engine using Couchbase as the backend database and Claude as the AI-powered embedding and language model provider. Semantic search goes beyond simple keyword matching by understanding the context and meaning behind the words in a query, making it an essential tool for applications that require intelligent information retrieval. This tutorial is designed to be beginner-friendly, with clear, step-by-step instructions that will equip you with the knowledge to create a fully functional semantic search system from scratch.

# Setting the Stage: Installing Necessary Libraries
To build our semantic search engine, we need a robust set of tools. The libraries we install handle everything from connecting to databases to performing complex machine learning tasks. Each library has a specific role: Couchbase libraries manage database operations, LangChain handles AI model integrations, and Claude provides advanced AI models for generating embeddings and understanding natural language. By setting up these libraries, we ensure our environment is equipped to handle the data-intensive and computationally complex tasks required for semantic search.

In [1]:
!pip install couchbase datasets langchain tqdm python-dotenv langchain-couchbase langchain-community langchain-anthropic langchain-openai openai

# Importing Necessary Libraries
The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading. These libraries provide essential functions for working with data, managing database connections, and processing machine learning models.

In [2]:
import json
import logging
import os
import time
import warnings
from datetime import timedelta
from uuid import uuid4

import numpy as np
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.exceptions import CollectionNotFoundException, CouchbaseException, InternalServerFailureException, QueryIndexAlreadyExistsException
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from datasets import load_dataset
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_core.documents import Document
from langchain_core.globals import set_llm_cache
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_couchbase.cache import CouchbaseCache
from langchain_couchbase.vectorstores import CouchbaseVectorStore
from langchain_openai import OpenAIEmbeddings
from tqdm import tqdm

# Setup Logging
Logging is configured to track the progress of the script and capture any errors or warnings. This is crucial for debugging and understanding the flow of execution. The logging output includes timestamps, log levels (e.g., INFO, ERROR), and messages that describe what is happening in the script.


In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configuring the Environment
Additionally, we'll load environment variables, which contains sensitive information like API keys and database credentials. Using environment variables instead of hardcoding these values into our script is a best practice that enhances security and flexibility, making it easier to manage and deploy our code in different environments.

In [4]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
CB_HOST = os.getenv('CB_HOST', 'couchbase://localhost')
CB_USERNAME = os.getenv('CB_USERNAME', 'Administrator')
CB_PASSWORD = os.getenv('CB_PASSWORD', 'password')
CB_BUCKET_NAME = os.getenv('CB_BUCKET_NAME', 'vector-search-testing')
INDEX_NAME = os.getenv('INDEX_NAME', 'vector_search_claude')

SCOPE_NAME = os.getenv('SCOPE_NAME', 'shared')
COLLECTION_NAME = os.getenv('COLLECTION_NAME', 'claude')
CACHE_COLLECTION = os.getenv('CACHE_COLLECTION', 'cache')

# Connecting to the Couchbase Cluster
Connecting to a Couchbase cluster is the foundation of our project. Couchbase will serve as our primary data store, handling all the storage and retrieval operations required for our semantic search engine. By establishing this connection, we enable our application to interact with the database, allowing us to perform operations such as storing embeddings, querying data, and managing collections. This connection is the gateway through which all data will flow, so ensuring it's set up correctly is paramount.



In [5]:
try:
    auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
    options = ClusterOptions(auth)
    cluster = Cluster(CB_HOST, options)
    cluster.wait_until_ready(timedelta(seconds=5))
    logging.info("Successfully connected to Couchbase")
except Exception as e:
    raise ConnectionError(f"Failed to connect to Couchbase: {str(e)}")

# Setting Up Collections in Couchbase
In Couchbase, data is organized in buckets, which can be further divided into scopes and collections. Think of a collection as a table in a traditional SQL database. Before we can store any data, we need to ensure that our collections exist. If they don't, we must create them. This step is important because it prepares the database to handle the specific types of data our application will process. By setting up collections, we define the structure of our data storage, which is essential for efficient data retrieval and management.

Moreover, setting up collections allows us to isolate different types of data within the same bucket, providing a more organized and scalable data structure. This is particularly useful when dealing with large datasets, as it ensures that related data is stored together, making it easier to manage and query.

In [6]:
try:
    bucket = cluster.bucket(CB_BUCKET_NAME)
    bucket_manager = bucket.collections()

    # Check if collection exists, create if it doesn't
    collections = bucket_manager.get_all_scopes()
    collection_exists = any(
        scope.name == SCOPE_NAME and COLLECTION_NAME in [col.name for col in scope.collections]
        for scope in collections
    )

    if not collection_exists:
        logging.info(f"Collection '{COLLECTION_NAME}' does not exist. Creating it...")
        bucket_manager.create_collection(SCOPE_NAME, COLLECTION_NAME)
        logging.info(f"Collection '{COLLECTION_NAME}' created successfully.")
    else:
        logging.info(f"Collection '{COLLECTION_NAME}' already exists.")

    # Wait for the collection to be available
    max_retries = 3
    retry_delay = 1
    for _ in range(max_retries):
        try:
            collection = bucket.scope(SCOPE_NAME).collection(COLLECTION_NAME)
            break
        except CollectionNotFoundException:
            time.sleep(retry_delay)
    else:
        raise RuntimeError(f"Collection '{COLLECTION_NAME}' not available after {max_retries} retries")

    # Ensure primary index exists
    try:
        cluster.query(f"CREATE PRIMARY INDEX ON `{CB_BUCKET_NAME}`.`{SCOPE_NAME}`.`{COLLECTION_NAME}`").execute()
        logging.info(f"Primary index created on collection '{COLLECTION_NAME}'")
    except QueryIndexAlreadyExistsException:
        logging.info(f"Primary index already exists on collection '{COLLECTION_NAME}'")
    except Exception as e:
        logging.warning(f"Error creating primary index: {str(e)}")

    # Clear all documents in the collection
    try:
        query = f"DELETE FROM `{CB_BUCKET_NAME}`.`{SCOPE_NAME}`.`{COLLECTION_NAME}`"
        cluster.query(query).execute()
        logging.info("All documents cleared from the collection.")
    except Exception as e:
        logging.warning(f"Error while clearing documents: {str(e)}. The collection might be empty.")
except Exception as e:
    raise RuntimeError(f"Error setting up collection: {str(e)}")


# Loading the Index Definition
Semantic search requires an efficient way to retrieve relevant documents based on a user’s query. This is where the Couchbase Full-Text Search (FTS) index comes in. An FTS index is designed to enable full-text search capabilities, such as searching for words or phrases within documents stored in Couchbase. In this step, we load the FTS index definition from a JSON file, which specifies how the index should be structured. This includes the fields to be indexed and other parameters that determine how the search engine processes queries. By defining an index, we ensure that our search engine can quickly and accurately retrieve the most relevant documents, which is crucial for delivering fast and relevant search results to users.


In [7]:
# index_definition_path = '/path_to_your_index_file/claude_index.json'

# Prompt user to upload to google drive
from google.colab import files
print("Upload your index definition file")
uploaded = files.upload()
index_definition_path = list(uploaded.keys())[0]

try:
    with open(index_definition_path, 'r') as file:
        index_definition = json.load(file)
except Exception as e:
    raise ValueError(f"Error loading index definition from {index_definition_path}: {str(e)}")

Upload your index definition file


Saving claude_index.json to claude_index (1).json


# Creating or Updating Search Indexes
With the index definition loaded, the next step is to create or update the FTS index in Couchbase. This step is fundamental because it optimizes our database for text search operations, allowing us to perform searches based on the content of documents rather than just their metadata. By creating or updating an FTS index, we make it possible for our search engine to handle complex queries that involve finding specific text within documents, which is essential for a robust semantic search engine.


In [8]:
try:
    scope_index_manager = cluster.bucket(CB_BUCKET_NAME).scope(SCOPE_NAME).search_indexes()

    # Check if index already exists
    existing_indexes = scope_index_manager.get_all_indexes()
    index_name = index_definition["name"]

    if index_name in [index.name for index in existing_indexes]:
        logging.info(f"Index '{index_name}' already exists. Updating...")
    else:
        logging.info(f"Creating new index '{index_name}'...")

    # Create SearchIndex object
    search_index = SearchIndex(
        name=index_definition["name"],
        source_type=index_definition.get("sourceType", "couchbase"),
        idx_type=index_definition["type"],
        source_name=index_definition["sourceName"],
        params=index_definition["params"],
        source_params=index_definition.get("sourceParams", {}),
        plan_params=index_definition.get("planParams", {})
    )

    # Upsert the index (create if not exists, update if exists)
    scope_index_manager.upsert_index(search_index)
    logging.info(f"Index '{index_name}' successfully created/updated.")
except QueryIndexAlreadyExistsException:
    logging.info(f"Index '{index_name}' already exists. Skipping creation/update.")
except InternalServerFailureException as e:
    logging.error(f"InternalServerFailureException raised: {str(e)}")
    raise RuntimeError(f"Internal server error while creating/updating search index: {str(e)}")
except Exception as e:
    raise RuntimeError(f"Unexpected error creating/updating search index: {str(e)}")


# Load the TREC Dataset
To build a search engine, we need data to search through. We use the TREC dataset, a well-known benchmark in the field of information retrieval. This dataset contains a wide variety of text data that we'll use to train our search engine. Loading the dataset is a crucial step because it provides the raw material that our search engine will work with. The quality and diversity of the data in the TREC dataset make it an excellent choice for testing and refining our search engine, ensuring that it can handle a wide range of queries effectively.

The TREC dataset's rich content allows us to simulate real-world scenarios where users ask complex questions, enabling us to fine-tune our search engine's ability to understand and respond to various types of queries.

In [9]:
try:
    trec = load_dataset('trec', split='train[:1000]')
    logging.info(f"Successfully loaded TREC dataset with {len(trec)} samples")
except Exception as e:
    raise ValueError(f"Error loading TREC dataset: {str(e)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Creating Claude Embeddings
Embeddings are at the heart of semantic search. They are numerical representations of text that capture the semantic meaning of the words and phrases. Unlike traditional keyword-based search, which looks for exact matches, embeddings allow our search engine to understand the context and nuances of language, enabling it to retrieve documents that are semantically similar to the query, even if they don't contain the exact keywords. By creating embeddings using Claude, we equip our search engine with the ability to understand and process natural language in a way that's much closer to how humans understand language. This step transforms our raw text data into a format that the search engine can use to find and rank relevant documents.



In [10]:
try:
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model='text-embedding-ada-002')
    logging.info("Successfully created OpenAIEmbeddings")
except Exception as e:
    raise ValueError(f"Error creating OpenAIEmbeddings: {str(e)}")

#  Setting Up the Couchbase Vector Store
A vector store is where we'll keep our embeddings. Unlike the FTS index, which is used for text-based search, the vector store is specifically designed to handle embeddings and perform similarity searches. When a user inputs a query, the search engine converts the query into an embedding and compares it against the embeddings stored in the vector store. This allows the engine to find documents that are semantically similar to the query, even if they don't contain the exact same words. By setting up the vector store in Couchbase, we create a powerful tool that enables our search engine to understand and retrieve information based on the meaning and context of the query, rather than just the specific words used.

In [11]:
try:
    vector_store = CouchbaseVectorStore(
        cluster=cluster,
        bucket_name=CB_BUCKET_NAME,
        scope_name=SCOPE_NAME,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        index_name=INDEX_NAME,
    )
    logging.info("Successfully created vector store")
except Exception as e:
    raise ValueError(f"Failed to create vector store: {str(e)}")


# Saving Data to the Vector Store
With the vector store set up, the next step is to populate it with data. We save the TREC dataset to the vector store in batches. This method is efficient and ensures that our search engine can handle large datasets without running into performance issues. By saving the data in this way, we prepare our search engine to quickly and accurately respond to user queries. This step is essential for making the dataset searchable, transforming raw data into a format that can be easily queried by our search engine.

Batch processing is particularly important when dealing with large datasets, as it prevents memory overload and ensures that the data is stored in a structured and retrievable manner. This approach not only optimizes performance but also ensures the scalability of our system.

In [12]:
batch_size = 50
try:
    for i in tqdm(range(0, len(trec['text']), batch_size), desc="Processing Batches"):
        batch = trec['text'][i:i + batch_size]
        documents = [Document(page_content=text) for text in batch]
        uuids = [str(uuid4()) for _ in range(len(documents))]
        vector_store.add_documents(documents=documents, ids=uuids)
except Exception as e:
    raise RuntimeError(f"Failed to save documents to vector store: {str(e)}")


Processing Batches: 100%|██████████| 20/20 [00:14<00:00,  1.39it/s]


# Setting Up a Couchbase Cache
To further optimize our system, we set up a Couchbase-based cache. A cache is a temporary storage layer that holds data that is frequently accessed, speeding up operations by reducing the need to repeatedly retrieve the same information from the database. In our setup, the cache will help us accelerate repetitive tasks, such as looking up similar documents. By implementing a cache, we enhance the overall performance of our search engine, ensuring that it can handle high query volumes and deliver results quickly.

Caching is particularly valuable in scenarios where users may submit similar queries multiple times or where certain pieces of information are frequently requested. By storing these in a cache, we can significantly reduce the time it takes to respond to these queries, improving the user experience.


In [13]:
try:
    cache = CouchbaseCache(
        cluster=cluster,
        bucket_name=CB_BUCKET_NAME,
        scope_name=SCOPE_NAME,
        collection_name=CACHE_COLLECTION,
    )
    logging.info("Successfully created cache")
    set_llm_cache(cache)
except Exception as e:
    raise ValueError(f"Failed to create cache: {str(e)}")

# Creating the OpenAi Language Model (LLM)
Language models are AI systems that are trained to understand and generate human language. We'll be using OpenAI's language model to process user queries and generate meaningful responses. This model is a key component of our semantic search engine, allowing it to go beyond simple keyword matching and truly understand the intent behind a query. By creating this language model, we equip our search engine with the ability to interpret complex queries, understand the nuances of language, and provide more accurate and contextually relevant responses.

The language model's ability to understand context and generate coherent responses is what makes our search engine truly intelligent. It can not only find the right information but also present it in a way that is useful and understandable to the user.



In [14]:
try:
    llm = ChatAnthropic(temperature=0, anthropic_api_key=ANTHROPIC_API_KEY, model_name='claude-3-5-sonnet-20240620')
    logging.info("Successfully created ChatAnthropic")
except Exception as e:
    logging.error(f"Error creating ChatAnthropic: {str(e)}. Please check your API key and network connection.")
    raise

# Building a Retrieval-Augmented Generation (RAG) Chain
A Retrieval-Augmented Generation (RAG) chain combines the power of retrieval (searching through data) with generation (producing a coherent response). This step integrates the vector store with the language model, enabling our search engine to retrieve relevant documents and generate well-formed answers to user queries. The RAG chain is what makes our search engine more intelligent, allowing it to not only find relevant information but also present it in a user-friendly manner. By building this chain, we create a system that can handle complex queries and provide users with meaningful, contextually appropriate responses.

The RAG chain bridges the gap between information retrieval and natural language understanding, creating a seamless experience where users can ask questions in plain language and receive detailed, relevant answers. This integration of retrieval and generation is what sets modern search engines apart, allowing them to provide much more than just a list of links or documents.

In [15]:
system_template = "You are a helpful assistant that answers questions based on the provided context."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "Context: {context}\n\nQuestion: {question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": lambda x: format_docs(vector_store.similarity_search(x)), "question": RunnablePassthrough()}
    | chat_prompt
    | llm
)
logging.info("Successfully created RAG chain")

# Executing a Sample Query and Performing Semantic Search
Finally, we bring everything together by executing a sample query. This step demonstrates the full power of the system we've built. The search engine processes the query, retrieves the most relevant documents from the vector store, and uses the language model to generate a coherent response. This step is where all the components of our system—Couchbase, OpenAi embeddings, the vector store, the cache, and the language model—work together to deliver a result that is not only accurate but also contextually appropriate.

This final step is a testament to the effectiveness of semantic search. It shows how our search engine can understand and respond to complex queries, providing users with the information they need in a format that's easy to understand. By combining advanced AI models with efficient database management, we've created a search engine that goes beyond traditional keyword-based search, offering a richer and more intuitive search experience.

In [16]:
query = "What caused the 1929 Great Depression?"

# Get responses
start_time = time.time()
rag_response = chain.invoke(query)
rag_elapsed_time = time.time() - start_time
logging.info(f"RAG response generated in {rag_elapsed_time:.2f} seconds")

print(f"RAG Response: {rag_response.content}")

# Perform semantic search
try:
    search_results = vector_store.similarity_search_with_score(query, k=10)
    results = [{'id': doc.metadata.get('id', 'N/A'), 'text': doc.page_content, 'distance': score}
               for doc, score in search_results]
    elapsed_time = time.time() - start_time
    logging.info(f"Semantic search completed in {elapsed_time:.2f} seconds")
except CouchbaseException as e:
    raise RuntimeError(f"Error performing semantic search: {str(e)}")

print(f"\nSemantic Search Results (completed in {elapsed_time:.2f} seconds):")
for result in results:
    print(f"Distance: {result['distance']:.4f}, Text: {result['text']}")


RAG Response: Based on the context provided, the world entered a global depression in 1929. This event is commonly known as "the Great Depression." While the context doesn't provide specific causes for the Great Depression, it's generally understood that it was triggered by a combination of factors, including:

1. The stock market crash of 1929
2. Bank failures
3. A decline in consumer spending and investment
4. International economic instabilities

It's important to note that the exact causes of the Great Depression are complex and have been debated by historians and economists. The context doesn't provide detailed information about these specific causes, but it does confirm that 1929 was the year when the global depression began.

Semantic Search Results (completed in 4.74 seconds):
Distance: 0.9177, Text: Why did the world enter a global depression in 1929 ?
Distance: 0.8714, Text: When was `` the Great Depression '' ?
Distance: 0.8116, Text: What crop failure caused the Irish Famin

By following these steps, you’ll have a fully functional semantic search engine that leverages the strengths of Couchbase and Claude. This guide is designed not just to show you how to build the system, but also to explain why each step is necessary, giving you a deeper understanding of the principles behind semantic search and how to implement it effectively. Whether you’re a newcomer to software development or an experienced developer looking to expand your skills, this guide will provide you with the knowledge and tools you need to create a powerful, AI-driven search engine.